In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import Word2Vec
import datetime
from dateutil.relativedelta import relativedelta

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df=pd.read_csv("new_data.csv")

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,5,6,7,8,9,13,14,15,16,18,20,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
df[df["user_id"]==75898][df["job_title"]=="Off Cycle Internship"][["job_id","job_title"]]

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,job_id,job_title


In [24]:
df['Timestamp'] =  pd.to_datetime(df['Timestamp'])
df['job_opening_date'] =  pd.to_datetime(df['job_opening_date'])
df['job_closing_date'] =  pd.to_datetime(df['job_closing_date'])

In [3]:
df=df[df.user_id.notnull()]

In [4]:
def get_all_users_id(df):
    users_id=df.user_id.values
    users_id=list(set(users_id))
    return users_id

In [5]:
all_users=get_all_users_id(df)

In [6]:
len(all_users)

165830

In [7]:
df=df[df.job_id.notnull()]

In [8]:
df.shape

(3799892, 32)

In [9]:
df.tail(1)

,Unnamed: 0,id,Timestamp,job_id,user_id,title,job_opening_date,job_closing_date,job_short_description,job_description,...,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,degree_subjectgroup_id,degree_subjectgroup_title
3799891,2875445,1356695.0,2018-08-31 12:13:05,2459.0,154137.0,Analyst Summer Internship 2019,2018-08-17,2018-10-21,Greenhill & Co. is a leading independent inves...,<h2>The role</h2>\r\r\n\r\r\n<p>Genuine analys...,...,False,2019.0,True,751.0,345.0,University of Cambridge,2.0,Geography,4.0,Geography


In [11]:
df.head(1)

,Unnamed: 0,id,Timestamp,job_id,user_id,title,job_opening_date,job_closing_date,job_short_description,job_description,...,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,degree_subjectgroup_id,degree_subjectgroup_title
0,0,1.0,2017-05-25 11:35:41,68.0,148463.0,Graduate Analyst,2017-04-24,NaN,We?re building the best consulting firm in the...,<h2>Overview</h2>\r\r\n\r\r\n<p>We&rsquo;ve cr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def get_users_id(df):
    users_id=df.user_id.values
    users_id=list(set(users_id))
    return users_id

In [13]:
users_id=get_users_id(df)

In [14]:
len(users_id)

30859

In [15]:
def get_jobs_id(df):
    jobs_id=df.job_id.values
    jobs_id=list(set(jobs_id))
    return jobs_id

In [16]:
jobs_id=get_jobs_id(df)

In [17]:
len(jobs_id)

4129

In [18]:
def get_jobs_per_user(users_id, df):
    n=len(users_id)
    print("n :",n)
    print("users collected")
    jobs_per_user={}
    for i,user in enumerate(users_id):
        if i%1000==0:
            print("progress :", i/n)
        x=df[df["user_id"]==user]
        x=x[x.job_id.notnull()]
        if x.shape[0]>0:
            job_applications=x["job_id"].values
            job_applications=list(set(job_applications))
            jobs_per_user[str(int(user))]=job_applications
    print("Jobs per user collected")
    return jobs_per_user

In [19]:
jobs_per_user=get_jobs_per_user(users_id, df)

n : 30859
users collected
progress : 0.0
progress : 0.0324054570789721
progress : 0.0648109141579442
progress : 0.0972163712369163
progress : 0.1296218283158884
progress : 0.1620272853948605
progress : 0.1944327424738326
progress : 0.22683819955280468
progress : 0.2592436566317768
progress : 0.29164911371074886
progress : 0.324054570789721
progress : 0.3564600278686931
progress : 0.3888654849476652
progress : 0.4212709420266373
progress : 0.45367639910560936
progress : 0.4860818561845815
progress : 0.5184873132635536
progress : 0.5508927703425257
progress : 0.5832982274214977
progress : 0.6157036845004699
progress : 0.648109141579442
progress : 0.6805145986584141
progress : 0.7129200557373861
progress : 0.7453255128163583
progress : 0.7777309698953304
progress : 0.8101364269743024
progress : 0.8425418840532746
progress : 0.8749473411322467
progress : 0.9073527982112187
progress : 0.9397582552901909
progress : 0.972163712369163
Jobs per user collected


In [26]:
def get_jobs_to_recommend(all_users,df,jobs_per_user):
    n=len(all_users)
    recommendations_per_user={}
    now = datetime.datetime.now()
    #d=df[df['job_created_date']>=now-relativedelta(years=1)]
    c=df[df['job_closing_date'].isnull()]
    d=df[df['job_closing_date']>=now]
    d=d[d.job_id.notnull()]
    jobs_to_apply_to=list(d["job_id"].values)+list(c["job_id"].values)
    jobs_to_apply_to=list(set(jobs_to_apply_to))
    print("number of jobs to apply to :",len(jobs_to_apply_to))
    print("recommendations started")
    for i,user in enumerate(all_users):
        if i%10000==0:
            print("progress :", i/n)
        try:
            job_applications=jobs_per_user[str(int(user))]
            recommendations=list(set(jobs_to_apply_to) - set(job_applications))
            recommendations_per_user[str(int(user))]=recommendations
        except:
            recommendations_per_user[str(int(user))]=jobs_to_apply_to
    return recommendations_per_user

In [27]:
recommendations_per_user=get_jobs_to_recommend(all_users, df, jobs_per_user)

number of jobs to apply to : 699
recommendations started
progress : 0.0
progress : 0.060302719652656336
progress : 0.12060543930531267
progress : 0.180908158957969
progress : 0.24121087861062535
progress : 0.30151359826328167
progress : 0.361816317915938
progress : 0.42211903756859437
progress : 0.4824217572212507
progress : 0.542724476873907
progress : 0.6030271965265633
progress : 0.6633299161792197
progress : 0.723632635831876
progress : 0.7839353554845323
progress : 0.8442380751371887
progress : 0.9045407947898451
progress : 0.9648435144425014


# create train and test dataset

In [30]:
a=list(jobs_per_user.keys())
train_user_id=[int(i) for i in a]
train_user_applications=list(jobs_per_user.values())

In [31]:
a=list(recommendations_per_user.keys())
test_user_id=[int(i) for i in a]
test_user_applications=list(recommendations_per_user.values())

In [32]:
class users:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.user_id=self.get_ids(d)
        self.length_of_course=get_length_of_course(d)
        self.gender=self.get_gender(d)
        self.ethnicity=self.get_ethnicity(d)
        self.course_includes_industrial_placement=self.get_course_includes_industrial_placement(d)
        self.graduation_year=self.get_graduation_year(d)
        self.right_to_work_uk=self.get_right_to_work_uk(d)
        self.degree_subject_id=self.get_degree_subject_id(d)
        self.university_id=self.get_university_id(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["user_id"]))
        return res
    
    def get_length_of_course(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["length_of_course"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_gender(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["gender"])
        return res
    
    def get_ethnicity(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["ethnicity"])
        return res
    
    def get_course_includes_industrial_placement(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["course_includes_industrial_placement"])
        return res
    
    def get_graduation_year(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["graduation_year"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_right_to_work_uk(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["right_to_work_uk"])
        return res
    
    def get_degree_subject_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["degree_subject_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_university_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["university_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["user_id"]=self.user_id
        dd["length_of_course"]=self.length_of_course
        dd["gender"]=self.gender
        dd["ethnicity"]=self.ethnicity
        dd["course_includes_industrial_placement"]=self.course_includes_industrial_placement
        dd["graduation_year"]=self.graduation_year
        dd["right_to_work_uk"]=self.right_to_work_uk
        dd["degree_subject_id"]=self.degree_subject_id
        dd["university_id"]=self.university_id
        self.df=pd.DataFrame(dd)